In [33]:
import type_handler

In [34]:
USE_TYPE_EFFECTIVENESS = True

We are only interested in damage, not in effects, so a move sums up to its name, its damage, its category (physical / special) and its type.

In [35]:
class Move:
    def __init__(self, name, damage, type, category):
        self.name = name
        self.damage = damage
        self.category = category
        self.type = type
    
    def calculate_damage(self, user, target): # Current formula.
        res = 2 * 100 / 5 + 2 
        res = res * self.damage * user.get_attack(self.category) / target.get_defense(self.category)
        res /= 50 
        res += 2 
        if user.has_type(self.type):
            res *= 1.5
        if USE_TYPE_EFFECTIVENESS:
            res *= type_handler.TYPE_HANDLER.coeff(self.type, target.types)
        return int(res)

    def __str__(self):
        return "{} dmg={} type={} cat={}".format(self.name, self.damage, self.category, self.type)

Similarly, we are only interested in damage, so a Pokémon will sum up to its name, typing, stats and movepool.
We use the modern formula for statistics, and we assume maximal EV and IV in the attacks if we consider an offensive Pokémon, and maximal EV and IV in the defenses if we consider a defensive Pokémon.

In [36]:
class Pokemon:
    DEFAULT_IV = 31
    DEFAULT_EV = 255
    DEFAULT_LEVEL = 100
    DEFAULT_NATURE = 1.0

    def __init__(self, name: str, form_name: str, number: int, form: int, stats: list, types: list):
        self.name = name
        self.form_name = form_name
        self.number = number
        self.form = form 
        self.stats = stats
        self.base_hp = stats[0]
        self.base_atk = stats[1]
        self.base_def = stats[2]
        self.base_speed = stats[3]
        self.base_spa = stats[4]
        self.base_spd = stats[5]
        self.types = types
        self.movepool = []
        self.damage_taken = 0
        self.damage_dealt = 0
    
    def set_movepool(self, list_moves):
        self.movepool = list_moves
    
    def reset(self):
        self.damage_taken = 0
        self.damage_dealt = 0
    
    def has_type(self, type):
        return type in self.types

    def get_hp(self):
        if self.base_hp == 1:
            return 1 
        res = 2 * self.base_hp + Pokemon.DEFAULT_IV + int(Pokemon.DEFAULT_EV / 4)
        res *= Pokemon.DEFAULT_LEVEL / 100
        res = int(res) + Pokemon.DEFAULT_LEVEL + 10
        return res 

    def get_attack(self, category):
        res = self.base_atk if category == 0 else self.base_spa
        res = 2 * res + Pokemon.DEFAULT_IV + int(Pokemon.DEFAULT_EV / 4)
        res *= Pokemon.DEFAULT_LEVEL / 100
        res = int(res) + 5
        res *= Pokemon.DEFAULT_NATURE
        return int(res)
    
    def get_defense(self, category):
        res = self.base_def if category == 0 else self.base_spd
        res = 2 * res + Pokemon.DEFAULT_IV + int(Pokemon.DEFAULT_EV / 4)
        res *= Pokemon.DEFAULT_LEVEL / 100
        res = int(res) + 5
        res *= Pokemon.DEFAULT_NATURE
        return int(res)
    
    def __str__(self):
        s = "{},{},{}".format(self.number, self.name, self.form_name)
        s += ",{},{},{},{},{},{}".format(self.base_hp, self.base_atk, self.base_def, self.base_spa, self.base_spd, self.base_speed)
        s += ",{},".format(self.types[0])
        if len(self.types) == 2:
            s += "{}".format(self.types[1])
        s += ",{},{}".format(self.damage_dealt, self.damage_taken)
        s += ",{},{}".format(self.get_hp(), self.damage_taken / self.get_hp())
        return s
    
    def is_same_poke(self, other):
        if self.name == other.name:
            return True
        
        base_name = self.name.split("_")[0]
        other_base_name = self.name.split("_")[0]
        
        if base_name != other_base_name:
            return False
        
        if self.types != other.types:
            return False
        
        if self.stats != other.stats:
            return False
        
        return True 

The following class loads everything (the data of the moves, of the Pokémons, of the forms). Typings are initialised in the type_handler file, since it's part of my fangame. 
The data comes from an old version of Pokémon Essentials (V18), more recent versions of the PBS files will probably fail to load since the formatting is different.

Then, use the run() or run2() functions to run the simulations: run() will simulate the damage from each Pokémon A to each Pokémon B using each move in its movepool, and run2() basically does the same but we can choose a fixed list of offensive Pokémons A (in my example, I use only Mew).

Finally, the result is logged into a file. To manipulate this data, use some spreadsheet software like Excel or Google Docs, split the data along the commas to separate the columns, and you can use the spreadsheet functions to order Pokémons.

Note that, while the initial video was about the bulk, this code also lets you evaluate the offensive capabilities of the Pokémons.

In [ ]:
class BulkScoreEngine:
    def __init__(self):
        self.all_pokemons = {}
        self.all_moves = {}
        self.ignored_pokes = ["DITTO", "UNOWN"]
        self.load_pokemons()
        self.load_movepool()
        self.load_pokemon_forms()
        self.load_movepool()
        self.load_moves()

    def load_pokemons(self):
        filename = "PBS\\pokemon.txt"
        name = ""
        form_name = ""
        number = 0
        form = 0
        stats = []
        types = []

        with open(filename, "r") as f:
            for line in f:
                line = line.strip()
                
                if line.startswith("#"):
                    continue 

                elif line.startswith("["):
                    if name != "":
                        poke = Pokemon(name, form_name, number, form, stats, types)
                        self.all_pokemons[name] = poke
                    
                    number = line.replace("[", "")
                    number = number.replace("]", "")
                    number = int(number)
                    name = ""
                    form = 0
                    stats = []
                    types = []
                elif line.startswith("InternalName"):
                    name = line.replace("InternalName = ", "")
                elif line.startswith("Type1"):
                    types.append(line.replace("Type1 = ", ""))
                elif line.startswith("Type2"):
                    types.append(line.replace("Type2 = ", ""))
                elif line.startswith("BaseStats"):
                    line = line.replace("BaseStats = ", "")
                    line_split = line.split(",")
                    stats = [int(s) for s in line_split]
        print("Loaded Pokémons.")

    def load_pokemon_forms(self):
        filename = "PBS\\pokemonforms.txt"
        name = ""
        base_name = ""
        form_name = ""
        number = 0
        form = 0
        stats = []
        types = []

        with open(filename, "r") as f:
            for line in f:
                line = line.strip()
                
                if line.startswith("#"):
                    continue 
                elif line.startswith("[BUTTERFREE,1]"):
                    # Don't include my forms.
                    break
                elif line.startswith("["):
                    if name != "":
                        if number == 0:
                            number = self.all_pokemons[base_name].number
                        if len(stats) == 0:
                            stats = self.all_pokemons[base_name].stats
                        if len(types) == 0:
                            types = self.all_pokemons[base_name].types

                        poke = Pokemon(name, form_name, number, form, stats, types)
                        if not self.all_pokemons[base_name].is_same_poke(poke):
                            self.all_pokemons[name] = poke
                    
                    line = line.replace("[", "")
                    line = line.replace("]", "")
                    line_split = line.split(",")

                    base_name = line_split[0]
                    name = base_name + "_" + line_split[1]
                    form = int(line_split[1])
                    form_name = ""

                    number = 0
                    stats = []
                    types = []
                
                elif line.startswith("FormName"):
                    form_name = line.replace("FormName = ", "")
                elif line.startswith("Type1"):
                    types.append(line.replace("Type1 = ", ""))
                    if types[-1] == "QMARKS":
                        name = ""
                elif line.startswith("Type2"):
                    types.append(line.replace("Type2 = ", ""))
                elif line.startswith("BaseStats"):
                    line = line.replace("BaseStats = ", "")
                    line_split = line.split(",")
                    stats = [int(s) for s in line_split]
        print("Loaded Pokémon forms.")

    def load_movepool(self):
        filename = "PBS\\sclearned.txt"
        with open(filename, "r") as f:
            for line in f:
                line = line.strip()

                if line.startswith("#"):
                    continue 

                line_split1 = line.split(" = ")
                poke_name = line_split1[0]

                if poke_name not in self.all_pokemons:
                    continue

                if poke_name in self.ignored_pokes:
                    continue
                
                if len(self.all_pokemons[poke_name].movepool) > 0:
                    continue

                line_split2 = line_split1[1].split(", ")
                movepool = [mv for mv in line_split2 if mv != line_split2[0]]
                self.all_pokemons[poke_name].set_movepool(movepool)
        print("Loaded movepools.")

    def load_moves(self):
        filename = "PBS\\moves.txt"
        with open(filename, "r") as f:
            for line in f:
                line = line.strip()
                
                if line.startswith("#"):
                    continue 
                if line.startswith("1000,"):
                    break

                line_split = line.split(",")
                name = line_split[1]
                damage = int(line_split[4])
                type = line_split[5]
                category = 2
                
                if line_split[6] == "Special":
                    category = 1
                elif line_split[6] == "Physical":
                    category = 0
                else:
                    # Don't add Status moves.
                    continue 

                self.all_moves[name] = Move(name, damage, type, category)
        print("Loaded moves.")

    def reset(self):
        for poke_name in self.all_pokemons:
            self.all_pokemons[poke_name].reset()

    def simulate_fight(self, attacking_pokemon, defending_pokemon, use_max = False):
        max_damage = 0
        for move_name in attacking_pokemon.movepool:
            if not move_name in self.all_moves:
                continue 
            move = self.all_moves[move_name]
            damage = move.calculate_damage(attacking_pokemon, defending_pokemon)

            if use_max:
                if damage > max_damage:
                    max_damage = damage
            else:
                defending_pokemon.damage_taken += damage
                attacking_pokemon.damage_dealt += damage
        if use_max:
            defending_pokemon.damage_taken += max_damage
            attacking_pokemon.damage_dealt += max_damage
    
    def run(self, use_max, attacking_poke_list = None):
        if not attacking_poke_list:
            attacking_poke_list = self.all_pokemons

        for defending_pokemon_name in self.all_pokemons:
            defending_pokemon = self.all_pokemons[defending_pokemon_name]

            for attacking_pokemon_name in attacking_poke_list:
                attacking_pokemon = self.all_pokemons[attacking_pokemon_name]

                # print("{} attacking {}".format(attacking_pokemon_name, defending_pokemon_name))

                self.simulate_fight(attacking_pokemon, defending_pokemon, use_max)

    def run2(self, attacking_poke_list):
        for defending_pokemon_name in self.all_pokemons:
            defending_pokemon = self.all_pokemons[defending_pokemon_name]

            for attacking_pokemon_name in attacking_poke_list:
                attacking_pokemon = self.all_pokemons[attacking_pokemon_name]

                # print("{} attacking {}".format(attacking_pokemon_name, defending_pokemon_name))

                self.simulate_fight(attacking_pokemon, defending_pokemon)
    
    def save_results(self, filename):
        with open(filename, "w") as f:
            f.write("Pokémon,HP,Atk,Def,SpAtk,SpDef,Speed,Type,Type,Damage Dealt,Damage Taken,HP,Num KO\n")
            for poke_name in self.all_pokemons:
                poke = self.all_pokemons[poke_name]
                f.write("{}\n".format(poke))
        



In [38]:
bce = BulkScoreEngine()

# Only Mew + with type effectiveness.
USE_TYPE_EFFECTIVENESS = True
bce.run(True, ["MEW"])
bce.save_results("results_mew_type_eff_max.txt")
bce.reset()
# Only Mew + without type effectiveness (purely stats).
USE_TYPE_EFFECTIVENESS = False
bce.run(True, ["MEW"])
bce.save_results("results_mew_max.txt")
bce.reset()
# All Pokémons + with type effectiveness.
USE_TYPE_EFFECTIVENESS = True
bce.run(True)
bce.save_results("results_type_eff_max.txt")
bce.reset()
# All Pokémons + without type effectiveness (purely stats).
USE_TYPE_EFFECTIVENESS = False
bce.run(True)
bce.save_results("results_max.txt")
bce.reset()


Loaded Pokémons.


AttributeError: 'BulkScoreEngine' object has no attribute 'ignored_pokes'

In [ ]:
# bce = BulkScoreEngine()

# Only Mew + with type effectiveness.
USE_TYPE_EFFECTIVENESS = True
bce.run(False, ["MEW"])
bce.save_results("results_mew_type_eff.txt")
bce.reset()
# Only Mew + without type effectiveness (purely stats).
USE_TYPE_EFFECTIVENESS = False
bce.run(False, ["MEW"])
bce.save_results("results_mew.txt")
bce.reset()
# All Pokémons + with type effectiveness.
USE_TYPE_EFFECTIVENESS = True
bce.run(False)
bce.save_results("results_type_eff.txt")
bce.reset()
# All Pokémons + without type effectiveness (purely stats).
USE_TYPE_EFFECTIVENESS = False
bce.run(False)
bce.save_results("results.txt")
bce.reset()


In [ ]:
bce.all_pokemons["BULBASAUR"].get_hp()